## Этап - *Парсинг и сохранение данных*

#### Импорт библиотек

Для парсинга, используя API Вконтакте, достаточно библиотеки requests. Для сохранения данных импортировал библиотеку csv.

In [ ]:
import requests
import csv

#### Парсинг и сохранение данных

Изучил необходимые методы из [документации](https://dev.vk.com/method) и, используя API Вконтакте, воспользовался соответствующими методами с заданными параметрами.

In [ ]:
def take_posts():
    '''
    Парсинг постов из группы vk.com/imct_fefu
    '''
    # Локальное расположение токена, чтобы скрыть его из репозитория
    with open('/Users/andrejgulaev/Documents/GitHub/Tokens/token-api.txt') as f:
        TOKEN = f.readline()
    VERSION = 5.131
    DOMAIN = 'imct_fefu'
    count = 100 # Указать кол-во постов. Кол-во может быть больше фактического. Кол-во смотреть здесь: https://vk.com/wall-206944280?offset=80&own=1

    response = requests.get('https://api.vk.com/method/wall.get',
                            params={
                                'access_token':TOKEN,
                                'v': VERSION,
                                'domain': DOMAIN,
                                'count': count
                        }
                        )
    data = response.json()['response']['items']
    return data

def file_writer(all_posts):
    '''
    Запись постов в табличный файл
    '''
    with open ('IMKT.csv', 'w') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('id', 'date_UNIX', 'date_MSK', 'likes', 'views', 'reposts', 'comments', 'describe'))
        for post in all_posts:
            date = datetime.fromtimestamp(post['date'] + (60*60*7))
            a_pen.writerow((post['id'],
                            post['date'],
                            date,
                            post['likes']['count'],
                            post['views']['count'],
                            post['reposts']['count'],
                            post['comments']['count'],
                            post['text']
                            ))

In [ ]:
all_posts = take_posts()
file_writer(all_posts)

In [ ]:
def take_comments():
    '''
    Парсинг комментариев с постов из группы vk.com/imct_fefu
    '''
    # Локальное расположение токена, чтобы скрыть его из репозитория
    with open('/Users/andrejgulaev/Documents/GitHub/Tokens/token-api.txt') as f:
        TOKEN = f.readline()
    VERSION = 5.131
    owner_id = -206944280
    count = 10
    post_id = 245 # ID последнего поста: https://vk.com/wall-206944280?offset=80&own=1
    all_comments = []
    while post_id > 0:
        response = requests.get('https://api.vk.com/method/wall.getComments',
                                params={
                                    'access_token':TOKEN,
                                    'v': VERSION,
                                    'owner_id': owner_id,
                                    'post_id': post_id,
                                    'count': count,
                                    'need_likes': 1
                            }
                        )
        post_id -= 1

        try:
            data = response.json()['response']['items']
        except:
            data = []
        all_comments.extend(data)
    return all_comments

def file_writer_comments(all_comments):
    '''
    Запись комментариев в табличный файл
    '''
    with open ('IMKT-comments.csv', 'w') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('id', 'from_id', 'likes', 'text'))
        for comment in all_comments:
            a_pen.writerow((comment['post_id'],
                            comment['from_id'],
                            comment['likes']['count'],
                            comment['text']
                            ))

In [ ]:
all_comments = take_comments()
file_writer_comments(all_comments)

In [ ]:
def take_members():
    '''
    Парсинг участников группы vk.com/imct_fefu
    '''
    # Локальное расположение токена, чтобы скрыть его из репозитория
    with open('/Users/andrejgulaev/Documents/GitHub/Tokens/token-api.txt') as f:
        TOKEN = f.readline()
    VERSION = 5.131
    group_id = 206944280
    count = 1000
    fields = 'sex, city'
    all_members = []
    response = requests.get('https://api.vk.com/method/groups.getMembers',
                            params={
                                'access_token':TOKEN,
                                'v': VERSION,
                                'group_id': group_id,
                                'count': count,
                                'fields': fields
                        }
                    )
    try:
        data = response.json()['response']['items']
    except:
        data = []
    all_members.extend(data)
    return all_members

def file_writer_members(all_members):
    with open ('IMKT-members.csv', 'w') as file:
        a_pen = csv.writer(file)
        a_pen.writerow(('id', 'first_name', 'last_name', 'sex', 'city'))
        for member in all_members:
            try:
                city = member['city']['title']
            except:
                city = np.NaN
            a_pen.writerow((member['id'],
                            member['first_name'],
                            member['last_name'],
                            member['sex'],
                            city
                            ))

In [ ]:
all_members = take_members()
file_writer_members(all_members)